In [ ]:
import requests
import pandas as pd
import numpy as np

### Set Up Mongo Connection

import urllib.parse
from pymongo import MongoClient

username = urllib.parse.quote_plus('w210_db_user')
password = urllib.parse.quote_plus('q1w2e3r4$')
conn = MongoClient('mongodb://%s:%s@198.11.212.212:27017/w210_db' % (username, password))
db = conn.w210_db

pd.set_option('display.max_columns', 500)

### Master List of ICOs

Get the master list of 2500+ ICOs to match.

In [ ]:
result = db['icotracker_ico'].find({})

data = []
for i,row in enumerate(result):
    data.append(row)
    
master = pd.DataFrame(data)
master['name_lower'] = master.apply(lambda x: str(x['name']).lower(),axis=1)
master.head()

In [ ]:
master.to_csv('master.csv')

In [ ]:
# Manipulate some columns

import datetime
def format_ico_dates(dates_text):
    dates = []
    for txt in dates_text.split(' '):
        try:
            valid_date = datetime.datetime.strptime(txt.strip(),'%d/%m/%Y')
            dates.append(datetime.datetime.strftime(valid_date,'%d/%m/%Y'))
        except ValueError as e:
            # Not a date, fail silently...
            pass
    if len(dates) == 1:
        dates = dates*2
    return ' - '.join(dates)

def ico_phase_ended(start_date):
    if start_date.strip() == '' or len(start_date.strip()) < 10:
        return 0
    elif datetime.datetime.strptime(start_date,'%d/%m/%Y') <= datetime.datetime.today():
        return 1
    else:
        return 0

master['key_dates'] = master.apply(lambda x: format_ico_dates(str(x['ico_dates'])),axis=1)
master['ico_phase_ended'] = master.apply(lambda x: ico_phase_ended(x['key_dates'].split(' - ')[-1].strip()),axis=1)
master.head()

### Source Various Exchange Datasets

In [ ]:
# GET request to CryptoCompare
response = requests.get("https://www.cryptocompare.com/api/data/coinlist/")
# Result comes back as a flat dict, transform to a list of dicts
data = []
for k,v in response.json()['Data'].items():
    data.append(v)
# Visualize as a dataframe 
df2 = pd.DataFrame(data)
df2.head()
# Create collection
db.sp_cryptocompare.remove()
db.sp_cryptocompare.insert_many(data)

In [ ]:
df2.head()

In [ ]:
# GET request to CryptoWatch
response = requests.get("https://api.cryptowat.ch/assets")
data = response.json()['result']
df3 = pd.DataFrame(data)
df3.head()

In [ ]:
deadcoin = db["deadcoin_ico"]
result = deadcoin.find({})

data = []
for i,row in enumerate(result):
    data.append(row)
df1 = pd.DataFrame(data)

In [ ]:
ico_prices = db["icostats_ico_price"]
result = deadcoin.find({})

data = []
for i,row in enumerate(result):
    data.append(row)
df4 = pd.DataFrame(data)
df4.head()

In [ ]:
# CoinMarketCap

from bs4 import BeautifulSoup
response = requests.get("https://coinmarketcap.com/all/views/all/")
soup = BeautifulSoup(response.text, "html5lib")
names = [x.text for x in soup.find_all("a", class_="currency-name-container link-secondary")]
df5 = pd.DataFrame(data=names,columns=['name'])

In [ ]:
# Manish 414

df6 = pd.read_csv('ICO_Reddit_Data.csv',delimiter='\t')
df6.head()

In [ ]:
len(df6)

In [5]:
db.db.collection_names()

['icorating_people_details',
 'deadcoin_ico',
 'icorating_people',
 'foo',
 'sp_overlap_analysis',
 'sp_cryptocompare',
 'icostats_ico_price',
 '414_ico_names',
 'crypto_compare_ico',
 'icotracker_ico',
 'ico_subreddit',
 'ico_social_media_stats',
 'reddit_summary',
 'icorating_ratings',
 'sp_price_history']

In [1]:
import pandas as pd
import numpy as np
import os
import sys
paths = {
    'helpers': '/home/spadela/mids-w210-capstone'
}
for k,v in paths.items():
    sys.path.insert(0, v)
    
from db import dbToolsMongo

In [2]:
db = dbToolsMongo()
df7 = db.toDF('414_ico_names')
df7.head()

,_id,name
0,5b46f5e565533a3db1b7dee1,A2BTaxiToken
1,5b46f5e565533a3db1b7dee2,adbank
2,5b46f5e565533a3db1b7dee3,AdEx
3,5b46f5e565533a3db1b7dee4,adshares
4,5b46f5e565533a3db1b7dee5,AdSigma


In [3]:
df7

,_id,name
0,5b46f5e565533a3db1b7dee1,A2BTaxiToken
1,5b46f5e565533a3db1b7dee2,adbank
2,5b46f5e565533a3db1b7dee3,AdEx
3,5b46f5e565533a3db1b7dee4,adshares
4,5b46f5e565533a3db1b7dee5,AdSigma
5,5b46f5e565533a3db1b7dee6,aeronaero
6,5b46f5e565533a3db1b7dee7,aeternity
7,5b46f5e565533a3db1b7dee8,agrello
8,5b46f5e565533a3db1b7dee9,Airpod_Sleeping_Pod
9,5b46f5e565533a3db1b7deea,AirToken


### Merge Function

Define a resusable merge function

In [ ]:
def merge_dataset(base,new,indicator,joinOn='name'):
    new[joinOn+'_lower'] = new[joinOn].apply(lambda x: str(x).lower())
    merged = pd.merge(base,new[[joinOn+'_lower']],left_on='name_lower',right_on=joinOn+'_lower',how='left',indicator=True)
    merged['ind_'+indicator.lower()] = merged['_merge'].apply(lambda x: 0 if x == 'left_only' else 1)
    merged = merged.drop(columns='_merge')
    
    return merged

datasets = [
    {'name': 'CoinMarketCap', 'data': df5, 'column': 'name'},
    {'name': 'CryptoCompare', 'data': df2, 'column': 'CoinName'},
    {'name': 'CryptoWatch', 'data': df3, 'column': 'name'},
    {'name': 'RavneetICOPrice', 'data': df4, 'column': 'name'}
]
df = master
for i,d in enumerate(datasets):
    print('Joining to %s on %s...' % (d['name'],d['column']))
    df = merge_dataset(base=df,new=d['data'],indicator=d['name'].lower(),joinOn=d['column'])

In [ ]:
df.head()

In [ ]:
flags = ['ico_phase_ended'] + ['ind_'+x['name'].lower() for x in datasets]
df.groupby(flags).aggregate('count')[['name_lower']]

### Notes

+ Only 21 ICOs match dead coin
+ **CryptoWatch** only yields an incremental 20 quotes
+ **CryptoCompare** matches roughly 25% of ICOs
+ **CoinMarketCap** matches roughly 20% of ICOs, but with a significant overlap to CryptoCompare

In [ ]:
cols = ['name'] + flags
data = df[cols].to_dict(orient='records')
db.sp_overlap_analysis.remove()
for row in data:
    db.sp_overlap_analysis.insert_one(row)

In [ ]:
result = db["sp_overlap_analysis"].find({})

data = []
for i,row in enumerate(result):
    data.append(row)
df1 = pd.DataFrame(data)
df1.head()

In [ ]:
df1.drop_duplicates(subset='name').sort_values(by=['name']).to_csv('overlap.csv')

In [ ]:
len(df1['name'].drop_duplicates())

In [ ]:
result = db['sp_price_history'].find({})

data = []
for i,row in enumerate(result):
    data.append(row)
    
data